In [ ]:
%matplotlib inline
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from data import *
from vimco import *
from plots import *
from torch.distributions.dirichlet import Dirichlet
import sys
import time
import datetime
sys.path.append('/home/hao/Research/probtorch/')
import probtorch
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

In [ ]:
T = 50
K = 4
D = 2

## Model Parameters
num_samples = 20
mcmc_steps = 8
num_particles_smc = 100
NUM_HIDDEN = 64
NUM_LATENTS = K*K
NUM_OBS = 2 * K
BATCH_SIZE = 20
NUM_EPOCHS = 1
LEARNING_RATE = 1e-3
CUDA = False

In [ ]:
Ys = torch.from_numpy(np.load('ball_dataset/sequences.npy')).float()
As_true = torch.from_numpy(np.load('ball_dataset/transitions.npy')).float()
Zs_true = torch.from_numpy(np.load('ball_dataset/states.npy')).float()
mus_true = torch.from_numpy(np.load('ball_dataset/means.npy')).float()
covs_true = torch.from_numpy(np.load('ball_dataset/covariances.npy')).float()
Pi = torch.from_numpy(np.load('ball_dataset/init.npy')).float()

prior = initial_trans_prior(K)

num_seqs = Ys.shape[0]

Ys_Zs = np.concatenate((Ys, Zs_true), axis=-1)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, num_obs=NUM_OBS,
                       num_hidden=NUM_HIDDEN,
                       num_latents=NUM_LATENTS):
        super(self.__class__, self).__init__()
        self.enc_hidden = nn.Sequential(
            nn.Linear(num_obs, num_hidden),
            nn.Tanh())
        self.latent_dir = nn.Sequential(
            nn.Linear(num_hidden, num_latents))
        
    def forward(self, obs, prior, batch_size):
        As = torch.zeros((batch_size, K, K))
        hidden = self.enc_hidden(obs)
        alphas = F.softmax(self.latent_dir(hidden), -1).view(batch_size, T-1, K*K).sum(1).view(batch_size, K, K) + prior
        for k in range(K):
            As[:, k, :] = Dirichlet(alphas[:, k, :]).sample()
        return alphas, As

In [ ]:
def initialize():
    enc = Encoder()
    if CUDA:
        enc.cuda()
    optimizer =  torch.optim.Adam(list(enc.parameters()),lr=LEARNING_RATE)    
    return enc, optimizer
enc, optimizer = initialize()

In [ ]:
ELBOs = []
ESSs = []
KLs = []
Grads = []

prior_mcmc = torch.ones((K, K)).float()
for k in range(K):
    prior_mcmc[k, k] = 3.0
    
Grad_Steps = int((Ys.shape[0] / BATCH_SIZE))
for epoch in range(NUM_EPOCHS):
    indices = torch.randperm(num_seqs)

    for step in range(Grad_Steps):
        time_start = time.time()
        optimizer.zero_grad()
        batch_indices = indices[step*BATCH_SIZE : (step+1)*BATCH_SIZE]
        batch_data = Ys[batch_indices]
        batch_zs = Zs_true[batch_indices]
        mu_ks = mus_true[batch_indices]
        cov_ks = covs_true[batch_indices]
        
        gradient, elbo, ess, kl = ag_mcmc_vimco_ball(enc, prior_mcmc, prior_mcmc, batch_zs, Pi, mu_ks, cov_ks, batch_data, T, D, K, num_samples, num_particles_smc, mcmc_steps, BATCH_SIZE)
        loss = - gradient
        loss.backward()
        optimizer.step()
        ELBOs.append(elbo.item())
        ESSs.append(ess.item())
        KLs.append(kl.item())
        Grads.append(loss.item())
        time_end = time.time()
        print('epoch : %d, step : %d, ELBO : %f, KL : %f (%ds)' % (epoch, step, elbo, kl, time_end - time_start))